# Optuna를 이용한 하이퍼파라미터 튜닝

Github 저장소: https://github.com/araffin/tools-for-robotic-rl-icra2022

Optuna: https://github.com/optuna/optuna

Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3

문서: https://stable-baselines3.readthedocs.io/en/master/

SB3 Contrib: https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

RL Baselines3 Zoo: https://github.com/DLR-RM/rl-baselines3-zoo

[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo)는 Stable-Baselines3를 사용해 사전 학습된 강화학습 에이전트들의 모음입니다.

또한 에이전트 훈련, 평가, 하이퍼파라미터 튜닝, 영상 녹화를 위한 기본 스크립트들도 제공합니다.


## 소개

이 노트북에서는 하이퍼파라미터 튜닝의 중요성을 배웁니다. 먼저 수동으로 파라미터를 최적화한 뒤, Optuna를 사용해 탐색을 자동화하는 방법을 알아봅니다.


## Pip을 이용한 의존성 및 Stable Baselines3 설치

전체 의존성 목록은 [README](https://github.com/DLR-RM/stable-baselines3)에서 확인할 수 있습니다.

```
pip install stable-baselines3[extra]
```

In [ ]:
!pip install stable-baselines3

In [ ]:
# 선택 사항: 추가 알고리즘 사용을 위해 SB3 Contrib 설치
!pip install sb3-contrib

In [ ]:
# 하이퍼파라미터 튜닝 마지막 단계에서 Optuna를 사용할 예정입니다
!pip install optuna

## Imports

In [ ]:
import gym
import numpy as np

가장 먼저 강화학습(RL) 모델을 임포트해야 하며, 어떤 문제에 어떤 모델을 사용할 수 있는지는 문서를 참고하세요.

In [ ]:
from stable_baselines3 import PPO, A2C, SAC, TD3, DQN

In [ ]:
# Contrib 저장소의 알고리즘들
# https://github.com/Stable-Baselines-Team/stable-baselines3-contrib
from sb3_contrib import QRDQN, TQC

In [ ]:
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

# 파트 I: 튜닝된 하이퍼파라미터의 중요성

지도 학습과 비교했을 때, 딥 강화학습은 학습률, 뉴런 수, 레이어 수, 옵티마이저 등과 같은 하이퍼파라미터의 선택에 훨씬 더 민감합니다.

하이퍼파라미터를 잘못 선택하면 수렴이 불안정하거나 성능이 저하될 수 있습니다. 이 문제는 네트워크 가중치와 환경 초기화를 위한 랜덤 시드에 따라 성능이 달라진다는 점에서 더 복잡해집니다.

하이퍼파라미터 외에도 적절한 알고리즘을 선택하는 것 역시 중요한 결정입니다. 간단한 Pendulum 태스크를 통해 이를 시연해보겠습니다.

[gym 문서](https://gym.openai.com/envs/Pendulum-v0/) 참고: "도립진자 스윙업 문제는 제어 이론에서 고전적인 문제입니다. 이 버전에서는 진자가 임의의 위치에서 시작하며, 목표는 진자를 스윙업하여 수직 상태를 유지하도록 하는 것입니다."

먼저 PPO 알고리즘과 4000 스텝(20 에피소드)이라는 작은 예산으로 시도해보겠습니다.

In [ ]:
env_id = "Pendulum-v1"
# 평가에만 사용되는 Env
eval_envs = make_vec_env(env_id, n_envs=10)
# 4000 스텝의 훈련 시간
budget_pendulum = 4000

### PPO

In [ ]:
ppo_model = PPO("MlpPolicy", env_id, seed=0, verbose=1).learn(budget_pendulum)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"PPO Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

### A2C

In [ ]:
# A2C 모델 정의 및 학습
a2c_model = A2C("MlpPolicy",env_id, seed=0, verbose=0).learn(budget_pendulum)

In [ ]:
# 학습된 A2C 모델 평가
mean_reward, std_reward = evaluate_policy(a2c_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"A2C Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

둘 다 환경을 해결하는 데는 거리가 멉니다(평균 보상 약 -200).  
이제 오프-폴리시 알고리즘을 시도해봅시다:

### PPO 더 오래 학습하기?

더 오래 학습하면 도움이 될까?

예산을 10배로 늘려서 시도해볼 수는 있지만, A2C/PPO의 경우에는 학습을 오래 한다고 해서 크게 나아지지 않는다. 대신 더 나은 하이퍼파라미터를 찾는 것이 필요하다.

In [ ]:
# 더 긴 학습
new_budget = 10 * budget_pendulum

ppo_model = PPO("MlpPolicy", env_id, seed=0, verbose=0).learn(new_budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"PPO Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

### PPO - 튜닝된 하이퍼파라미터

Optuna를 사용하면, 실제로 하이퍼파라미터를 튜닝하여 작동하는 솔루션을 찾을 수 있습니다([RL Zoo](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/hyperparams/ppo.yml) 참고):

In [ ]:
tuned_params = {
    "gamma": 0.9,
    "use_sde": True,
    "sde_sample_freq": 4,
    "learning_rate": 1e-3,
}

# budget = 10 * budget_pendulum
ppo_tuned_model = PPO("MlpPolicy", env_id, seed=1, verbose=1, **tuned_params).learn(50_000, log_interval=5)

In [ ]:
mean_reward, std_reward = evaluate_policy(ppo_tuned_model, eval_envs, n_eval_episodes=100, deterministic=True)

print(f"Tuned PPO Mean episode reward: {mean_reward:.2f} +/- {std_reward:.2f}")

참고: 간단한 MountainCarContinuous 환경에서 SAC를 시도하면, 하이퍼파라미터를 튜닝하지 않으면 몇 가지 문제에 직면할 수 있다: https://github.com/rail-berkeley/softlearning/issues/76

간단한 환경조차도 최신 알고리즘(SOTA)에게는 도전적일 수 있다.

# 파트 II: 대학원생 하강법

### 챌린지 (10분): "대학원생 하강법"  
이 챌린지의 목표는 제한된 예산(20,000 학습 스텝) 내에서 `CartPole-v1` 환경에서 A2C 알고리즘의 최대 성능을 끌어낼 수 있는 최적의 하이퍼파라미터를 찾는 것이다.  

`CartPole-v1`에서의 최대 보상: 500  

하이퍼파라미터는 서로 다른 랜덤 시드에서도 잘 작동해야 한다.

In [ ]:
budget = 20_000

#### 기준선: 기본 하이퍼파라미터

In [ ]:
eval_envs_cartpole = make_vec_env("CartPole-v1", n_envs=10)

In [ ]:
model = A2C("MlpPolicy", "CartPole-v1", seed=8, verbose=1).learn(budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, eval_envs_cartpole, n_eval_episodes=50, deterministic=True)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

**목표는 해당 기준선을 넘어서고 최적 점수인 500에 더 가까워지는 것입니다**

튜닝할 시간입니다!

In [ ]:
import torch.nn as nn

In [ ]:
policy_kwargs = dict(
    net_arch=[
      dict(vf=[64, 64], pi=[64, 64]), # 액터/크리틱을 위한 네트워크 아키텍처
    ],
    activation_fn=nn.Tanh,
)

hyperparams = dict(
    n_steps=5, # 정책 업데이트 전 데이터를 수집할 단계 수
    learning_rate=7e-4,
    gamma=0.99, # 할인 계수
    max_grad_norm=0.5, # 그래디언트 클리핑의 최댓값
    ent_coef=0.0, # 손실 계산을 위한 엔트로피 계수
)

model = A2C("MlpPolicy", "CartPole-v1", seed=8, verbose=1, **hyperparams).learn(budget)

In [ ]:
mean_reward, std_reward = evaluate_policy(model, eval_envs_cartpole, n_eval_episodes=50, deterministic=True)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

힌트 - 권장 하이퍼파라미터 범위

```python
gamma = trial.suggest_float("gamma", 0.9, 0.99999, log=True)
max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
# 2**3 = 8에서 2**10 = 1024까지
n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
# net_arch tiny: {"pi": [64], "vf": [64]}
# net_arch default: {"pi": [64, 64], "vf": [64, 64]}
# activation_fn = nn.Tanh / nn.ReLU
```

# 파트 III: 자동 하이퍼파라미터 튜닝

이 파트에서는 최적의 하이퍼파라미터를 자동으로 탐색할 수 있는 스크립트를 생성할 것입니다.

### Imports

In [ ]:
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances

### Config

In [ ]:
N_TRIALS = 100  # 최대 트라이얼 수
N_JOBS = 1 # 병렬로 실행할 작업 수
N_STARTUP_TRIALS = 5  # N_STARTUP_TRIALS 후 무작위 샘플링 중지
N_EVALUATIONS = 2  # 학습 중 평가 횟수
N_TIMESTEPS = int(2e4)  # 학습 예산
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_ENVS = 5
N_EVAL_EPISODES = 10
TIMEOUT = int(60 * 15)  # 15분

ENV_ID = "CartPole-v1"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpPolicy",
    "env": ENV_ID,
}

### 실습 (5분): 탐색 공간 정의하기

참고: https://github.com/optuna/optuna-examples/blob/main/rl/sb3_simple.py

In [ ]:
from typing import Any, Dict
import torch
import torch.nn as nn

def sample_a2c_params(trial: optuna.Trial) -> Dict[str, Any]:
    """
    A2C 하이퍼파라미터 샘플러

    :param trial: Optuna 트라이얼 객체
    :return: 주어진 트라이얼에 대한 샘플링된 하이퍼파라미터
    """
    # 0.9~0.9999 사이의 할인율
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    # 8, 16, 32, ... 1024
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)

    ### 여기에 코드를 입력하세요
    # TODO:
    # - 학습률 탐색 공간 정의하기 [1e-5, 1] (log) -> `suggest_float`
    # - 네트워크 아키텍처 탐색 공간 정의하기 ["tiny", "small"] -> `suggest_categorical`
    # - 활성화 함수 탐색 공간 정의하기 ["tanh", "relu"]
    learning_rate = ...
    net_arch = ...
    activation_fn = ...

    ### 코드 끝

    # 실제 값 표시
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]}
        if net_arch == "tiny"
        else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]

    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "learning_rate": learning_rate,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
        },
    }

### 목적 함수를 정의하세요

먼저 Optuna에 주기적인 평가 결과를 보고하는 커스텀 콜백을 정의합니다:

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback

class TrialEvalCallback(EvalCallback):
    """
    트라이얼 평가 및 보고에 사용되는 콜백입니다.

    :param eval_env: 평가 환경
    :param trial: Optuna 트라이얼 객체
    :param n_eval_episodes: 평가 에피소드 수
    :param eval_freq:   콜백이 ``eval_freq``번 호출될 때마다 에이전트를 평가
    :param deterministic: 평가가 확률적 정책을 사용해야 하는지, 결정적 정책을 사용해야 하는지 여부
    :param verbose: 출력 없음의 경우 0, 정보 메시지의 경우 1, 디버그 메시지의 경우 2
    """

    def __init__(
        self,
        eval_env: gym.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):

        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            # 정책 평가(부모 클래스에서 수행)
            super()._on_step()
            self.eval_idx += 1
            # Optuna에 보고서 보내기
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # 필요한 경우 가지치기 시도
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True

### 실습 (10분): 목적 함수 정의하기

그런 다음 하이퍼파라미터를 샘플링하고 모델을 생성한 후 그 결과를 Optuna로 반환하는 목적 함수를 정의합니다.

In [ ]:
def objective(trial: optuna.Trial) -> float:
    """
    Optuna에서 평가에 사용하는 목적 함수입니다.

    하나의 설정(즉, 하나의 하이퍼파라미터 세트)을 평가합니다.

    트라이얼 객체가 주어지면 하이퍼파라미터를 샘플링하고,
    평가한 후 결과를 보고합니다(학습 후 평균 에피소드 보상).

    :param trial: Optuna 트라이얼 객체
    :return: 학습 후 평균 에피소드 보상
    """

    kwargs = DEFAULT_HYPERPARAMS.copy()
    ### 여기에 코드 작성
    # TODO:
    # 1. 하이퍼파라미터를 샘플링하고 기본 키워드 인수를 업데이트합니다: `kwargs.update(other_params)`
    # 2. 평가 환경을 생성합니다.
    # 3. `TrialEvalCallback`을 생성합니다.
    # 1. 하이퍼파라미터를 샘플링하고 키워드 인수를 업데이트합니다.

    # RL 모델 생성


    # 2. `make_vec_env`, `ENV_ID`, `N_EVAL_ENVS`를 사용하여 평가에 사용될 환경을 생성합니다.

    
    # 3. 위에 정의된 `TrialEvalCallback` 콜백을 생성하여 `EVAL_FREQ`마다 `N_EVAL_EPISODES`를 사용하여 주기적으로 평가하고
    # 성능을 보고합니다.
    # TrialEvalCallback 시그니처:
    # TrialEvalCallback(eval_env, trial, n_eval_episodes, eval_freq, deterministic, verbose)
    eval_callback = ...

    ### 코드 끝

    nan_encountered = False
    try:
        # 모델 학습
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # 때때로 무작위 하이퍼파라미터는 NaN을 생성할 수 있습니다.
        print(e)
        nan_encountered = True
    finally:
        # 메모리 해제
        model.env.close()
        eval_envs.close()

    # 옵티마이저에게 트라이얼이 실패했음을 알립니다.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward


### 최적화 루프

In [ ]:
import torch as th

# 더 빠른 학습을 위해 pytorch num threads를 1로 설정
th.set_num_threads(1)
# 샘플러 선택, random, TPESampler, CMAES 등 가능
sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
# 최대 예산의 1/3을 사용하기 전에는 가지치기하지 않음
pruner = MedianPruner(
    n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3
)
# 스터디를 생성하고 하이퍼파라미터 최적화 시작
study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")

try:
    study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS, timeout=TIMEOUT)
except KeyboardInterrupt:
    pass

print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print(f"  Value: {trial.value}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

print("  User attrs:")
for key, value in trial.user_attrs.items():
    print(f"    {key}: {value}")

# 보고서 작성
study.trials_dataframe().to_csv("study_results_a2c_cartpole.csv")

fig1 = plot_optimization_history(study)
fig2 = plot_param_importances(study)

fig1.show()
fig2.show()

전체 코드 예시: https://github.com/DLR-RM/rl-baselines3-zoo

# 결론

이 노트북에서 우리는 다음과 같은 내용을 살펴보았습니다:
- 좋은 하이퍼파라미터의 중요성  
- Optuna를 활용한 자동 하이퍼파라미터 탐색 방법